In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [34]:
locG_a1_temps = pd.read_excel("Country G.xlsx", sheet_name=1, skiprows=3).dropna().reset_index(drop=True)
locG_a1_temps['Date'] = datetime(1970,1,1)
locG_a1_temps['Avg_Temp'] = np.nan
for i, row in locG_a1_temps.iterrows():
    yr, mt, d, maxt, mint, dt, avgt = row.values
    locG_a1_temps.loc[i, 'Date'] = datetime(int(yr), int(mt), int(d))
    locG_a1_temps.loc[i, 'Avg_Temp'] = (maxt + mint)/2
locG_a1_temps['doy'] = locG_a1_temps.Date.apply(lambda x: x.day_of_year)
locG_a1_temps = locG_a1_temps.rename(columns={'Year':'year', 'max ':'Max_Temp', 'min':'Min_Temp'}).drop("Day",axis=1)
locG_a1_temps

,year,Month,Max_Temp,Min_Temp,Date,Avg_Temp,doy
0,1991.0,1.0,27.8,22.7,1991-01-01,25.25,1
1,1991.0,1.0,27.8,21.3,1991-01-02,24.55,2
2,1991.0,1.0,27.8,21.9,1991-01-03,24.85,3
3,1991.0,1.0,28.1,22.2,1991-01-04,25.15,4
4,1991.0,1.0,27.8,21.7,1991-01-05,24.75,5
...,...,...,...,...,...,...,...
10953,2020.0,12.0,27.8,22.8,2020-12-27,25.30,362
10954,2020.0,12.0,28.0,20.8,2020-12-28,24.40,363
10955,2020.0,12.0,29.0,21.4,2020-12-29,25.20,364
10956,2020.0,12.0,27.8,22.4,2020-12-30,25.10,365


In [38]:
locG_a1_rain = pd.read_excel("Country G.xlsx", sheet_name=2, skiprows=4).dropna().reset_index(drop=True)
locG_a1_rain['Date'] = datetime(1970,1,1)
for i, row in locG_a1_rain.iterrows():
    yr, mt, d, rain, dt = row.values
    locG_a1_rain.loc[i, 'Date'] = datetime(int(yr), int(mt), int(d))
locG_a1_rain

,Year,Month,Day,Rainfall,Date
0,1991,1,1,3.9,1991-01-01
1,1991,1,2,3.1,1991-01-02
2,1991,1,3,2.9,1991-01-03
3,1991,1,4,2.3,1991-01-04
4,1991,1,5,3.3,1991-01-05
...,...,...,...,...,...
10953,2020,12,27,2.7,2020-12-27
10954,2020,12,28,0.1,2020-12-28
10955,2020,12,29,0.0,2020-12-29
10956,2020,12,30,3.3,2020-12-30


In [46]:
df = locG_a1_temps.merge(locG_a1_rain[['Rainfall']], left_index=True, right_index=True, how='left')
df['year'] = df.year.astype(int)
df['Month'] = df.Month.astype(int)
df['area'] = 1
df['country'] = "G"
df = df.rename(columns={'Rainfall':"rain"})[["doy","Month","year","rain","Date","Avg_Temp","Min_Temp","Max_Temp","area","country"]]
df

,doy,Month,year,rain,Date,Avg_Temp,Min_Temp,Max_Temp,area,country
0,1,1,1991,3.9,1991-01-01,25.25,22.7,27.8,1,G
1,2,1,1991,3.1,1991-01-02,24.55,21.3,27.8,1,G
2,3,1,1991,2.9,1991-01-03,24.85,21.9,27.8,1,G
3,4,1,1991,2.3,1991-01-04,25.15,22.2,28.1,1,G
4,5,1,1991,3.3,1991-01-05,24.75,21.7,27.8,1,G
...,...,...,...,...,...,...,...,...,...,...
10953,362,12,2020,2.7,2020-12-27,25.30,22.8,27.8,1,G
10954,363,12,2020,0.1,2020-12-28,24.40,20.8,28.0,1,G
10955,364,12,2020,0.0,2020-12-29,25.20,21.4,29.0,1,G
10956,365,12,2020,3.3,2020-12-30,25.10,22.4,27.8,1,G


In [47]:
df.to_parquet("CountryG_Area1_cleaned_merged.parquet")